# Calculate Intersections

## Overview

This notebook calculates the number of interesections between new power plant sitings from the GODEEEP multi-model coupling experiment and various areas of interest. The areas of interest explored include: Disadvantaged communities, important farmland, and areas within and within close proximity to composite natural areas in the western US. The number of interesections indicate how many square km of new energy infrastructure reside in each of these areas. We conduct this calculation for new infrastructure under both the net zero (NZ) and business-as-usual (BAU) scenarios through 2050. Intersections are determined at the state, technology, and state + technology level. Lastly, the difference in intersections between the two scenarios is also determined.

## Data Requirements

This notebook relies on the data prepared in the `prepare_raster_data.ipynb` and `prepare_cerf_siting_output.ipynb` notebooks in this repository. Those notebooks have been pre-run for convenience and the data files referenced in this notebook are found in the `data/input_data` folder.

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
from shapely import Point

import rasterio
from rasterio.plot import show

import os
from pathlib import Path

### Collect Data Paths

In [ ]:
year = 2050

# data dir
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'input_data')

# output data dir
output_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'output_data')
os.makedirs(output_dir, exist_ok=True)

# infrastructure siting output csv
infrastucture_path = os.path.join(data_dir, 'infrastructure_data_csv', f'infrastructure_data.csv')

# land area files
land_area_raster_dir = os.path.join(data_dir, "west_raster_data", 'land_area')

# all area files
all_area_raster_dir = os.path.join(data_dir, "west_raster_data", 'all_area')

# dac analysis files
cg_dac_path = os.path.join(output_dir,  f'net_zero_dac_analysis_{year}.csv')
bau_dac_path = os.path.join(output_dir, f'bau_dac_analysis_{year}.csv')
diff_dac_path = os.path.join(output_dir, f'difference_dac_analysis_{year}.csv')

# farmland dir
cg_farm_path = os.path.join(output_dir, f'net_zero_farm_analysis_{year}.csv')
bau_farm_path = os.path.join(output_dir, f'bau_farm_analysis_{year}.csv')
diff_farm_path = os.path.join(output_dir, f'difference_farm_analysis_{year}.csv')

# environmental dir
cg_env_path = os.path.join(output_dir, f'net_zero_env_analysis_{year}.csv')
bau_env_path = os.path.join(output_dir, f'bau_env_analysis_{year}.csv')
diff_env_path = os.path.join(output_dir, f'difference_env_analysis_{year}.csv')


### Functions

In [ ]:
def results_to_geodataframe(df, crs = "ESRI:102003"):
    """ 
    Takes a pandas DataFrame with x and y coordinates as input and 
    converts to a GeoPandas GeodataFrame. Coordinates in DataFrame are expected to
    follow the ESRI:102003 albers equal area conic coordinate referece system.
    x-coordinate column should be called 'xcoord', y-coordinate column should be
    called 'ycoord'

    :param df:        input Pandas DataFrame with x/y coordinates
    :type df:         Pandas DataFrame
    
    :param crs:       Coordinate reference system to use for GeoDataFrame
    :type crs:        str
    
    """
    
    # create geometry column from coordinate fields
    geometry = [Point(xy) for xy in zip(df['xcoord'], df['ycoord'])]
    
    gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return gdf

def extract_coords(df):
    """
    Collects a list of x/y coordinates from a dataframe as a list of tuples.
    
    :param df:        input Pandas DataFrame with x/y coordinates
    :type df:         Pandas DataFrame
    
    """

    # extract x,y coords to list of tuples
    coords = [(x,y) for x, y in zip(df['xcoord'], df['ycoord'])]
    
    return coords


def analyze_intersections(df, raster_list, layer_name_dict):
    """
    Reads in a pandas DataFrame of power plant sitings and calculates the number of sitings that intersect
    with the provided rasters given in the raster_list praameter. Output is a dictionary of interaction counts by
    technology, area, and state.

    :param df:                    input Pandas DataFrame with x/y coordinates of power plant sitings
    :type df:                     Pandas DataFrame
    
    :param raster_list:           list of paths to raster files to assess
    :type raster_list:            list

    :param layer_name_dict:       dictionary of file name and plain language name pairs
    :type layer_name_dict:        dict    
    
    """
    
    # dictionary to store outputs in
    d = {'state': [], 'technology': [], 'technology_simple': [], 'layer_name': [], 'layer': [], 'total_plants':[], 'intersection': []}

    for state in list(df.region_name.unique()):
        
        state_df = df[df.region_name == state]
    
        for tech in list(state_df.technology.unique()):
            
            total_plants = 0
    
            n_conflict_plants = 0
            
            # reduce dataframe to technology type
            df_tech = state_df[state_df.technology == tech]

            # simple tech name
            simple_tech = list(df_tech.technology_simple.unique())[0]

            # extract the coordinates from the dataframe
            coords = extract_coords(df_tech)

            # total number of power plants for the CONUS
            total_plants = len(coords)

            # update suitability for each power plant once it is within a grid cell that is unsuitable
            for raster in raster_list:

                with rasterio.open(raster) as src:

                    # extract the suitability value (0 == suitable; 1 == unsuitable) for each power plant
                    extract_suitability_at_points = [int(i[0]) for i in src.sample(coords)]
    
                    # get the count of unsuitable designations
                    n_conflict_plants = len([i for i in extract_suitability_at_points if i == 1])
    
                    # append the results to the output dictionary
                    d['state'].append(state)
                    d['technology'].append(tech)
                    d['technology_simple'].append(simple_tech)
                    d['layer_name'].append(layer_name_dict[os.path.basename(raster)])
                    d['layer'].append(os.path.basename(raster.lower()))
                    d['total_plants'].append(total_plants)
                    d['intersection'].append(n_conflict_plants)

    # convert the output to a pandas dataframe
    layer_specific_diagnostics = pd.DataFrame(d)

    layer_specific_diagnostics['fraction'] = round(layer_specific_diagnostics['intersection'] / layer_specific_diagnostics['total_plants'], 2)

    layer_sort_list=[]
    for layer in layer_name_dict:
        layer_sort_list.append(layer)

    layer_specific_diagnostics.layer = pd.Categorical(layer_specific_diagnostics.layer, categories=layer_sort_list)
    layer_specific_diagnostics=layer_specific_diagnostics.sort_values('layer')
    
    return layer_specific_diagnostics.sort_values(by='technology')


def calculate_intersection_difference(nz_analysis_df, bau_analysis_df):
    """
    Reads in results dataframe for each scenario and calculates the difference in intersections by technology, state, and area of
    interest. Returns a pandas DataFrame of the differences by technology, state, and area of interest.

    :param nz_analysis_df:            Dataframe of net zero intersection results
    :type nz_analysis_df:             Pandas DataFrame
    
    :param bau_analysis_df:           Dataframe of business-as-usual intersection results
    :type bau_analysis_df:            Pandas DataFrame
    
    """
    

    # rename intersection columns to scenario name
    nz_analysis_df = nz_analysis_df.rename(columns={'intersection':'net_zero'})[['state', 'technology','layer', 'total_plants', 'net_zero']]                                                                                           
    bau_analysis_df = bau_analysis_df.rename(columns={'intersection':'bau'})[['state', 'technology', 'layer','total_plants', 'bau']]
    
    # combine dataframes
    combined_analysis_df = pd.merge(nz_analysis_df, bau_analysis_df, how='left', on=['state', 'technology', 'layer'])

    # fill nan values with 0
    combined_analysis_df['bau'].fillna(0, inplace=True)
    combined_analysis_df['net_zero'].fillna(0, inplace=True)

    # calculate the difference in land
    combined_analysis_df['intersection'] = combined_analysis_df['net_zero'] - combined_analysis_df['bau']

    # convert to integer
    combined_analysis_df['intersection'] = combined_analysis_df['intersection'].astype(int)

    # simplify output dataframe
    combined_analysis_df = combined_analysis_df[['state', 'technology', 'layer', 'net_zero', 'bau', 'intersection']]

    return combined_analysis_df


# Analysis

#### Collect Infrastructure Data

In [ ]:
# collect prepared CERF infrastructure siting data
df = pd.read_csv(infrastucture_path)

#### Prepare Power Plant Sitings

In [ ]:
# reduce to only include sitings up to year of interest
df = df[df.sited_year <= year]

# create a dataframe of cerf power plant sitings for the clean grid scenario
df_clean_grid = df[(df.scenario == 'net_zero_ira_ccs_climate') & (df.cerf_sited == 1)].copy()

# create a dataframe of cerf power plant sitings for the business-as-usual scenario
df_bau = df[(df.scenario == 'business_as_usual_ira_ccs_climate') & (df.cerf_sited == 1)].copy()

In [ ]:
# dictionary of DAC layers with their simple names
dac_dict = {'gridcerf_usceq_cejst_exclude_all_dacs.tif': 'DACs',
            'gridcerf_usceq_cejst_exclude_energy_dacs.tif': 'Energy DACs',
            'gridcerf_usceq_cejst_exclude_transportation_dacs.tif': 'Transportation DACs',
            'gridcerf_usceq_cejst_exclude_housing_dacs.tif':'Housing DACs',
            'gridcerf_usceq_cejst_exclude_pollution_dacs.tif':'Pollution DACs',
            'gridcerf_usceq_cejst_exclude_health_dacs.tif':'Health DACs',
            'gridcerf_usceq_cejst_exclude_climate_dacs.tif':'Climate DACs',
            'gridcerf_usceq_cejst_exclude_workforce_dacs.tif':'Workforce DACs',
            'gridcerf_usceq_cejst_exclude_water_dacs.tif':'Water DACs'}

# dictionary of important farmland layers with their simple names
farm_dict = {'gridcerf_usda_combined_important_farmland.tif': 'Important Farmland',
            'gridcerf_usda_nrsc_prime_farmland_classification.tif': 'Prime Farmland',
            'gridcerf_usda_nrsc_farmland_of_state_importance.tif': 'Farmland of State Importance',
            'gridcerf_usda_nrsc_farmland_of_local_importance.tif':'Farmland of Local Importance',
            'gridcerf_usda_nrsc_farmland_of_unique_importance.tif':'Farmland of Unique Importance'}

# dictionary of environmental layers with their simple names
env_dict = {'gridcerf_combined_environment_level_1_1buff.tif': 'Within 1 km of Natural Area',
           'gridcerf_combined_environment_level_1_5buff.tif': 'Within 5 km of Natural Area',
           'gridcerf_combined_environment_level_1_10buff.tif': 'Within 10 km of Natural Area',
           }

## Disadvantaged Communities

<b> Scientific Question Addressed </b>: Under decarbonization and business-as-usual scenarios, how much land required for new power plant infrastructure intersects with federally-identified disadvantaged communities?

In [ ]:
# create a list of paths to the DAC layers
dac_raster_list = []
for layer in dac_dict:
    layer_name = dac_dict[layer]
    layer_path = os.path.join(all_area_raster_dir, layer)
    dac_raster_list.append(layer_path)

#### Clean Grid - New Power Plant Sitings

In [ ]:
# settings
input_df = df_clean_grid.copy()
raster_list = dac_raster_list
layer_dict = dac_dict
analysis_type = 'DAC Layers'
scenario = f'Net Zero by {year}'
output_file = cg_dac_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the DAC layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

#### Business-as-usual - New Power Plant Sitings

In [ ]:
# settings
input_df = df_bau.copy()
raster_list = dac_raster_list
layer_dict = dac_dict
analysis_type = 'DAC Layers'
scenario = f'Business-as-usual by {year}'
output_file = bau_dac_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the DAC layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

______________

## Important Farmland

In [ ]:
# create a list of paths to the farmland layers
farm_raster_list = []
for layer in farm_dict:
    layer_name = farm_dict[layer]
    layer_path = os.path.join(all_area_raster_dir, layer)
    farm_raster_list.append(layer_path)

#### **Net Zero - New Power Plant Sitings**

In [ ]:
# settings
input_df = df_clean_grid.copy()
raster_list = farm_raster_list
layer_dict = farm_dict
analysis_type = 'Important Farmland Layers'
scenario = f'Net Zero by {year}'
output_file = cg_farm_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the farmland layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

#### **Business-as-usual - New Power Plant Sitings**

In [ ]:
# settings
input_df = df_bau.copy()
raster_list = farm_raster_list
layer_dict = farm_dict
analysis_type = 'Important Farmland Layers'
scenario = f'Business-as-usual by {year}'
output_file = bau_farm_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the DAC layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

_________

## Natural Area Proximity

<b> Scientific Question Addressed </b>: Under decarbonization and business-as-usual scenarios, how much land required for new power plant infrastructure intersects with areas in close proximity to natural areas?

In [ ]:
# create a list of paths to the DAC layers
env_raster_list = []
for layer in env_dict:
    layer_name = env_dict[layer]
    layer_path = os.path.join(all_area_raster_dir, layer)
    env_raster_list.append(layer_path)

#### Clean Grid - New Power Plant Sitings

In [ ]:
# settings
input_df = df_clean_grid.copy()
raster_list = env_raster_list
layer_dict = env_dict
analysis_type = 'Natural Area Proximity'
scenario = f'Net Zero by {year}'
output_file = cg_env_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the DAC layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

#### **Business-as-usual - New Power Plant Sitings**

In [ ]:
# settings
input_df = df_bau.copy()
raster_list = env_raster_list
layer_dict = env_dict
analysis_type = 'Natural Area Proximity'
scenario = f'Business-as-usual by {year}'
output_file = bau_env_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the DAC layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

___________ 

# Difference Between Scenarios Analysis
This section calculates the additional land intersections between the Net Zero and Business-as-usual scenario.

## Disadvantaged Communities

In [ ]:
# clean grid settings
input_df = df_clean_grid.copy()
raster_list = dac_raster_list
layer_dict = dac_dict
analysis_type = 'Disadvantaged Communities'
scenario = f'Net Zero by {year}'

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
clean_grid_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

# bau settings
input_df = df_bau.copy()
raster_list = dac_raster_list
layer_dict = dac_dict
analysis_type = 'Disadvantaged Communities'
scenario = f'Business-as-usual by {year}'

output_file = diff_dac_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
bau_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Calculating difference in power plant intersections for {analysis_type}...')
# calculate the intersection difference between scenarios
analysis_df = calculate_intersection_difference(clean_grid_analysis_df, bau_analysis_df)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

_______

## Important Farmland

In [ ]:
# clean grid settings
input_df = df_clean_grid.copy()
raster_list = farm_raster_list
layer_dict = farm_dict
analysis_type = 'Important Farmland'
scenario = f'Net Zero by {year}'

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
clean_grid_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

# bau settings
input_df = df_bau.copy()
raster_list = farm_raster_list
layer_dict = farm_dict
analysis_type = 'Important Farmland'
scenario = f'Business-as-usual by {year}'

output_file = diff_farm_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
bau_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Calculating difference in power plant intersections for {analysis_type}...')
# calculate the intersection difference between scenarios
analysis_df = calculate_intersection_difference(clean_grid_analysis_df, bau_analysis_df)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

__________

## Natural Area Proximity


In [ ]:
# clean grid settings
input_df = df_clean_grid.copy()
raster_list = env_raster_list
layer_dict = env_dict
analysis_type = 'Natural Area Proximity'
scenario = f'Net Zero by {year}'

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
clean_grid_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

# bau settings
input_df = df_bau.copy()
raster_list = env_raster_list
layer_dict = env_dict
analysis_type = 'Natural Area Proximity'
scenario = f'Business-as-usual by {year}'

output_file = diff_env_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
bau_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Calculating difference in power plant intersections for {analysis_type}...')
# calculate the intersection difference between scenarios
analysis_df = calculate_intersection_difference(clean_grid_analysis_df, bau_analysis_df)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

### How Many 2020 Power Plants Intersect with DACs?

In [ ]:
# collect 2020 power plant locations
prev_df = df[df.scenario =='net_zero_ira_ccs_climate'].copy()
prev_df = prev_df[prev_df.cerf_sited == 0]
prev_df = prev_df[prev_df.sited_year<2021]

# extract their coordinates
coords = extract_coords(prev_df)


raster = os.path.join(all_area_raster_dir, 'gridcerf_usceq_cejst_exclude_all_dacs.tif')
with rasterio.open(raster) as src:

    # extract the suitability value (0 == suitable; 1 == unsuitable) for each power plant
    extract_suitability_at_points = [int(i[0]) for i in src.sample(coords)]

    # get the count of unsuitable designations
    n_conflict_plants = len([i for i in extract_suitability_at_points if i == 1])

print(f'{n_conflict_plants} square km of power plant infrastructure intersects with DACs in 2020.')